# 트리의 앙상블

In [1]:
# 정형 데이터를 다루는데 가장 뛰어난 성과를 내는 알고리즘
# 비정형 데이터 : 신경망 알고리즘, 비정형데이터는 규칙성을 찾기어려워서 머신러닝으로는 모델을 만들기가 어려움

## 랜덤포레스트

In [2]:
# 랜덤포레스트
# 결정 트리를 랜덤하게 만들어서 결정트리인 숲으로 만든다.
# 각 결정 트리의 예측을 사용하여 최종예측을 만든다

# 각 트리를 훈련하기 위한 데이터를 랜덤하게 추출
# 훈련 데이터에서 랜덤하게 샘플 추출, 샘플 중복 추출가능 : 부트스트랩
# 기본적으로 부트스트랩 샘플은 훈련 데이터 세트의 크기를 같게만든다
# 선택되지 않는 샘플도 있을수 있다

# 각 노드를 분할할때 전체 특성 중에서 일부 특성을 현명하게 선택하고 이중에서 최선의 분할을 찾는다
# RandomForestClassifier는 기본 특성개수의 제곱근 만큼의 특성을 선택 : 4개의 특성일경우 2가지 특성을 선택

# 희귀 모델인 RandomForestRegressor는 전체특성 사용

# scikit-Learn의 RandomForest는 기본적으로 100개의 결정트리를 위와같은 방식으로 훈련
# 분류일 경우 : 각 트리의 클래스별 확률을 평균하여 가장 높은 확률을 가진 클래스를 예측으로 사용
# 회귀의 경우 : 단순히 각 트리의 예측을 평균함

# 랜덤하게 선택한 샘플과 특성을 사용하므로 훈련세트에 과적합 되는것을 방지하고
# 검증 및 테스트 세트에서 안정적 성능을 얻을 수 있음.

In [3]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

wine = pd.read_csv("./wine.csv")

data = wine[["alcohol", "sugar", "pH"]].to_numpy()
target = wine["class"].to_numpy()
train_input, test_input, train_target, test_target = train_test_split(data, target, test_size = 0.2, random_state = 42)

In [4]:
from sklearn.model_selection import cross_validate
from sklearn.ensemble import RandomForestClassifier

rf = RandomForestClassifier(n_jobs=-1, random_state=42)
scores = cross_validate(rf, train_input, train_target, return_train_score=True, n_jobs=-1)
# return_train_score=True : 검증점수와 훈련점수도 함께 리턴, 과대적합 파악에 용이
print(np.mean(scores["train_score"]), np.mean(scores["test_score"]))

0.9973541965122431 0.8905151032797809


In [5]:
# RandomForest는 DecisionTree의 ensemble이므로 DecisionTreeClassifier인 모든 매개변수 사용 가능
# 결정 트리의 장점인 특성 중요도를 계산하고, 각 결정트리의 특성 중요도 취함
rf.fit(train_input, train_target)
print(rf.feature_importances_)

[0.23167441 0.50039841 0.26792718]


In [6]:
# 자체적으로 모델 평가 점수 획득 가능
# 부트스트랩 샘플에 포함되지않고 남은 샘플(OOB : out of bag)을 사용하여
# 부트스트랩 샘플로 훈련한 결정 트리를 평가할 수있다(검증 세트와 같은 역할)
rf = RandomForestClassifier(oob_score=True, n_jobs=-1, random_state=42)
rf.fit(train_input, train_target)
print(rf.oob_score_)

0.8934000384837406


## 엑스트라트리

In [7]:
# 부트스트랩 샘플을 사용하지않음, 결정트리를 만들때 전체훈련 세트 사용
# 노드를 분할할때는 가장 좋읁 분할을 찾는게 아닌 무작위로 분할
# 결정트리를 하나면 사용하고 무작위 분할할 경우 성능이 낮아지지만 많은 트리를 앙상블하므로
# 과대 적합을 방지하고 검증세트의 정수를 높일 수 있음

In [8]:
from sklearn.ensemble import ExtraTreesClassifier
et = ExtraTreesClassifier(n_jobs=-1, random_state=42)
scores = cross_validate(et, train_input, train_target, return_train_score = True, n_jobs=-1)
print(np.mean(scores["train_score"]), np.mean(scores["test_score"]))

0.9974503966084433 0.8887848893166506


In [9]:
et.fit(train_input, train_target)
print(et.feature_importances_)

[0.20183568 0.52242907 0.27573525]
